## Simulating any compartmental model using the `Spp` class

In the present notebook, we show how (most) compartmenal models can be simulated using the `pyross.deterministic.Spp` class. We allow users to specify any number of epidemiological classes, as well as any linear or infectious coupling between them.

In [ ]:
%%capture
## compile PyRoss for this notebook
import os
owd = os.getcwd()
os.chdir('../../')
%run setup.py install
os.chdir(owd)

In [ ]:
%matplotlib inline
import numpy as np
import pyross
import matplotlib.pyplot as plt
#from matplotlib import rc; rc('text', usetex=True)

### The SIR model

Below you will find the model-specification dictionary for the SIR model:

In [ ]:
model_spec = {
    "classes" : ["S", "I"],

    "S" : {
        "linear"    : [],
        "infection" : [ ["I", "-beta"] ]
    },

    "I" : {
        "linear"    : [ ["I", "-gamma"] ],
        "infection" : [ ["I", "beta"] ]
    }
}



This corresponds to

$$
\begin{aligned}
\dot{S}_i & = - \beta \sum_\nu C_{ij} \frac{I_j}{N_j} S_i \\
\dot{I}_i & = \beta \sum_\nu C_{ij} \frac{I_j}{N_j} - \gamma I_i \\
\dot{R}_i & = \gamma I_i^\mu
\end{aligned}
$$

Let's go through each component of the model specification step-by-step:

-  The list `"classes" : ["S", "I", "R"]` defines the epidemiological
classes of the model. <i>The order in which they are written are important</i>, as this ordering will have to be mainained if giving the initial conditions of the simulation as an array. Each model requires the presence of a susceptible class. This class
will always be the first element of the list `classes`, regardless of whether it is labelled as `S` or not.
- The dynamics of each class is defined by a key-value pair. Consider

    <br>

    ```json
    "E" : {
        "linear"    : [ ["E", "-gammaE"] ],
        "infection" : [ ["I", "betaI"], ["A", "betaA"] ]
    },
    ```
    
    <br>
    
  - This reads out as:
      $$\dot{E}^\mu = -\gamma_E E + \beta_I \sum_\nu C^I_{\mu \nu} \frac{I^\nu}{N^\nu} S^\mu + \beta_A \sum_\nu C^A_{\mu \nu} \frac{A^\nu}{N^\nu} S^\mu.$$
  - The linear terms for each epidemic class is defined by the lists of lists:
  
    <br>
    
    ```json
    "linear"    : [ ["E", "-gammaE"] ]
    ```
    
    <br>
    
    Eeach pair in `linear` corresponds to the linear coupling 
    with the class and the coupling constant respectively. So
    `["E", "-gammaE"]` corresponds to the term $-\gamma_E E$ in
    the equation for $\dot{E}$. The minus sign in front of `gammaE`
    signifies that the negative of the coefficient should be used.
  - The infection terms are defined in a similar manner. Each pair
    in `infection` corresponds to the non-linear coupling with $S$
    and the coupling constant respectively. So `["I", "betaI"]`
    corresponds to the term $\beta_I \sum_\nu C^I_{\mu \nu} \frac{I^\nu}{N^\nu} S$.

Next, we define the parameter values:

In [ ]:
parameters = {
    'beta' : 0.3,
    'gamma' : 0.1
}

The initial conditions can be defined in either of two ways. They can either be defined using a dictionary, where for each model class we have a corresponding 1D array of length $M$ (where $M$ is the number of age-groups), or a numpy array. The numpy array must have dimensions $M \times (d-1)$, where $d$ is the number of model classes (so 3 for SIR, for example).

If the initial conditions are provided as a dictionary, we are free to leave out one of the classes. In which case the initial conditions of the left out class will be inferred from the others.

In [ ]:
# Initial conditions as an array

x0 = np.array([
    999, 1000, 1000, # S
    1,   0,    0,    # I
])

# Initial conditions as a dictionary

x0 = {
    # S will be inferred from I and R
    'I' : [1, 0, 0],
    'R' : [0, 0, 0]
}

In [ ]:
M = 3                
Ni = 1000*np.ones(M)   
N = np.sum(Ni) 

CM = np.array([
    [1,   0.5, 0.1],
    [0.5, 1,   0.5],
    [0.1, 0.5, 1  ]
], dtype=float)

def contactMatrix(t):   
    return CM

# duration of simulation and data file
Tf = 160;  Nt=160; 

model = pyross.deterministic.Spp(model_spec, parameters, M, Ni)

# simulate model 
data = model.simulate(x0, contactMatrix, Tf, Nt)

In [ ]:
# plot the data and obtain the epidemic curve
S = np.sum(model.model_class_data('S', data), axis=1)
I = np.sum(model.model_class_data('I', data), axis=1)
R = np.sum(model.model_class_data('R', data), axis=1)
t = data['t']

fig = plt.figure(num=None, figsize=(10, 8), dpi=80, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 22})

plt.fill_between(t, 0, S/N, color="#348ABD", alpha=0.3)
plt.plot(t, S/N, '-', color="#348ABD", label='$S$', lw=4)

plt.fill_between(t, 0, I/N, color='#A60628', alpha=0.3)
plt.plot(t, I/N, '-', color='#A60628', label='$I$', lw=4)

plt.fill_between(t, 0, R/N, color="dimgrey", alpha=0.3)
plt.plot(t, R/N, '-', color="dimgrey", label='$R$', lw=4)

plt.legend(fontsize=26); plt.grid() 
plt.autoscale(enable=True, axis='x', tight=True)
plt.ylabel('Fraction of compartment value')
plt.xlabel('Days');

### SEAIR model

Let us now look at a more complicated example.

In [ ]:
model_spec = {
    "classes" : ["S", "E", "A", "Ia", "Is"],

    "S" : {
        "linear"    : [],
        "infection" : [ ["A", "-betaA"], ["Ia", "-betaIa"], ["Is", "-betaIs"] ]
    },

    "E" : {
        "linear"    : [ ["E", "-gammaE"] ],
        "infection" : [ ["A", "betaA"], ["Ia", "betaIa"], ["Is", "betaIs"] ]
    },
    
    "A" : {
        "linear"    : [ ["E", "gammaE"], ["A", "-alphabar*gammaA"], ["A", "-alpha*gammaA"] ],
        "infection" : [ ]
    },

    "Ia" : {
        "linear"    : [ ["A", "alpha*gammaA"],["Ia", "-gammaIa"] ],
        "infection" : [ ]
    },
    
    "Is" : {
        "linear"    : [ ["A", "alphabar*gammaA"], ["Is", "-gammaIs"] ],
        "infection" : [ ]
    }
}

gammaA = 0.2
alpha = 0              # fraction of asymptomatic infectives

parameters = {
    'betaA' : 0.2,
    'betaIa' : 0.2,
    'betaIs' : 0.2,
    'gammaE' : 0.04,
    'alpha*gammaA' : alpha*gammaA,
    'alphabar*gammaA' : (1 - alpha)*gammaA,
    'gammaIa' : 0.1,
    'gammaIs' : 0.1,
}

This corresponds to

$$
\begin{aligned}
\dot{S}_i & = - \lambda_i(t) S_i  \\
\dot{E}_i & = \lambda_i(t) S_i - \gamma_E E_i \\
\dot{A}_i & = \gamma_E E_i - \gamma_A A_i \\
\dot{I^a}_i & =  \alpha \gamma_A A  - \gamma_I^a I^a_i \\
\dot{I^s}_i & =  (1 - \alpha)\gamma_A A  - \gamma_I^s I^s_i \\
\dot{R}_i & = \gamma_I^a I^a_i + \gamma_I^s I^s_i
\end{aligned}
$$

where

$$
\lambda_i(t) = \sum_j C_{ij} \left( \beta_I^a  \frac{I^a_j}{N_j} + \beta_I^s  \frac{I^s_j}{N_j} + \beta_A  \frac{A_j}{N_j}\right).
$$


$$
C_{ij}  \beta_I^a  \frac{I^a_j}{N_j}
$$

In [ ]:
M = 3                
Ni = 10000*np.ones(M)   
N = np.sum(Ni)        

x0 = {
    'E' : np.array([0,0,0]),
    'A' : np.array([1,0,0]),
    'Ia' : np.array([0,0,0]),
    'Is' : np.array([0,0,0]),
    'R' : np.array([0,0,0])
}

CM = np.array([
    [1,   0.5, 0.1],
    [0.5, 1,   0.5],
    [0.1, 0.5, 1  ]
], dtype=float)

def contactMatrix(t):   
    return CM

# duration of simulation and data file
Tf = 300;  Nt=161; 

model = pyross.deterministic.Spp(model_spec, parameters, M, Ni)

# simulate model 
data = model.simulate(x0, contactMatrix, Tf, Nt)

In [ ]:
# plot the data and obtain the epidemic curve

t = data['t']

fig = plt.figure(num=None, figsize=(10, 8), dpi=80, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 22})

for model_class in (model_spec['classes'] + ['R'] ):
    Os = np.sum(model.model_class_data(model_class, data), axis=1)
    plt.plot(t, Os/N, '-', label='$%s$' % model_class, lw=4)

plt.legend(fontsize=26); plt.grid() 
plt.autoscale(enable=True, axis='x', tight=True)
plt.ylabel('Fraction of compartment value')
plt.xlabel('Days');